<a href="https://colab.research.google.com/github/wedingdong/sentiment-analysis-chatgpt-indobert-tfidf/blob/main/UAS_NLP_Pipeline_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Tahap 0: Setup (Instalasi & Import Library)**

In [ ]:
# --- Tahap 0: Setup (Instalasi & Import Library) ---

# 1. Instalasi Library
# Menginstal semua paket eksternal yang dibutuhkan untuk proyek.
!pip install transformers datasets accelerate scikit-learn pandas gradio -q

# 2. Import Library
# Pustaka standar untuk manipulasi data dan komputasi
import pandas as pd
import numpy as np
import torch
import os

# Pustaka untuk antarmuka (UI) demo
import gradio as gr

# Pustaka untuk menghubungkan Google Drive
from google.colab import drive

# Pustaka scikit-learn untuk membagi data dan metrik evaluasi
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, classification_report

# Pustaka Hugging Face Transformers untuk model NLP
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    Trainer,
    TrainingArguments,
    pipeline
)
from datasets import Dataset

print("--- Tahap 0 Selesai: Semua library terinstal dan ter-import. ---")

--- Tahap 0 Selesai: Semua library terinstal dan ter-import. ---


# **Tahap 1: Pemuatan Data Mentah (Data Loading)**

In [ ]:
# --- Tahap 1: Pemuatan & Pembersihan Data ---

# 1. Hubungkan (Mount) Google Drive
# Kode ini memicu otorisasi untuk mengakses file di Google Drive.
print("--- Menghubungkan ke Google Drive... ---")
try:
    drive.mount('/content/drive')
    print("--- Google Drive Terhubung. ---")
except Exception as e:
    if "already mounted" in str(e):
        print("--- Google Drive sudah terhubung. ---")
    else:
        print(f"Error saat mount Drive: {e}")

# 2. Tentukan Path (Lokasi) File
FILE_PATH = "/content/drive/MyDrive/UAS_NLP/Datasets_ulasan_chatgpt_BIndo.csv"

# 3. Muat & Bersihkan Data Mentah
if not os.path.exists(FILE_PATH):
    print(f"ERROR: File TIDAK DITEMUKAN di path '{FILE_PATH}'")
else:
    print(f"File ditemukan di {FILE_PATH}. Memuat data...")

    try:
        # Percobaan 1: Standar UTF-8
        df = pd.read_csv(FILE_PATH, on_bad_lines='skip', encoding='utf-8')
    except UnicodeDecodeError:
        print("Encoding UTF-8 gagal. Mencoba encoding alternatif (latin1)...")
        # Percobaan 2: Fallback ke Latin-1
        df = pd.read_csv(FILE_PATH, on_bad_lines='skip', encoding='latin1')

    print(f"Baris data awal (mentah): {len(df)}")

    # --- 4. Perbaikan Kritis untuk Data Kotor ---
    # Memaksa kolom 'score' menjadi format angka.
    # 'errors='coerce'' mengubah teks/tanggal yang salah masuk ke 'score' menjadi NaN.
    df['score'] = pd.to_numeric(df['score'], errors='coerce')

    # 5. Pembersihan Data Inti
    # Membuang baris jika content kosong atau score tidak valid (NaN).
    df.dropna(subset=['content', 'score'], inplace=True)
    print(f"Baris data setelah pembersihan (siap dipakai): {len(df)}")

    # Menampilkan 5 baris pertama data yang sudah bersih
    print("\nContoh data bersih:")
    print(df.head())

--- Menghubungkan ke Google Drive... ---
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
--- Google Drive Terhubung. ---
File ditemukan di /content/drive/MyDrive/UAS_NLP/Datasets_ulasan_chatgpt_BIndo.csv. Memuat data...
Encoding UTF-8 gagal. Mencoba encoding alternatif (latin1)...
Baris data awal (mentah): 14993
Baris data setelah pembersihan (siap dipakai): 13194

Contoh data bersih:
                               reviewId         userName  \
0  ba1d3668-3790-4e05-b6ee-75d1ae5ed82d  Pengguna Google   
2  c36e6ecd-f458-46dd-bea0-e90f6d10bd5e  Pengguna Google   
4  9a0568d5-f903-413d-8c7b-7f0e7ee20b0e  Pengguna Google   
5  9848a4a2-4b6b-41ef-9d66-a5c262bcdeb7  Pengguna Google   
6  683c30e8-5a9a-4085-97b9-ad2b9f89b99b  Pengguna Google   

                                           userImage  \
0  https://play-lh.googleusercontent.com/EGemoI2N...   
2  https://play-lh.googleusercontent.com/EGemoI2N...   
4 

# **Tahap 2: Annotation (Label Engineering) dan Balancing**

In [ ]:
# --- Tahap 2: Anotasi Hibrida & Penyeimbangan Data (Strategi Data Besar) ---

from sklearn.utils import resample

# 1. Definisikan Kamus Sentimen (Lexicon) Super Lengkap
# Diperluas untuk menangkap nuansa performa, akurasi, dan harga pada ChatGPT.
KAMUS_POSITIF = [
    # Umum & Emosi
    'bagus', 'suka', 'keren', 'puas', 'mantap', 'cinta', 'senang', 'takjub',
    'hebat', 'luar biasa', 'sempurna', 'the best', 'best', 'good', 'nice',
    'top', 'oke', 'sip', 'gokil', 'gacor', 'rekomen', 'juara', 'love',
    # Fungsionalitas
    'canggih', 'pintar', 'cerdas', 'akurat', 'tepat', 'jenius', 'solutif',
    'membantu', 'bermanfaat', 'berguna', 'informatif', 'edukatif', 'paham',
    'mengerti', 'jelas', 'detail', 'lengkap', 'kreatif', 'inovatif',
    # Performa
    'cepat', 'kilat', 'lancar', 'mudah', 'gampang', 'simpel', 'praktis',
    'ringan', 'responsif'
]

KAMUS_NEGATIF = [
    # Umum & Emosi
    'jelek', 'buruk', 'parah', 'kecewa', 'nyesel', 'benci', 'sampah', 'ampas',
    'bad', 'worst', 'gagal', 'payah', 'bodoh', 'tolol', 'aneh', 'kacau',
    # Masalah Teknis
    'error', 'bug', 'lemot', 'lambat', 'lelet', 'lag', 'ngelag', 'macet',
    'hang', 'crash', 'keluar sendiri', 'force close', 'berat', 'loading lama',
    'lama', 'susah', 'ribet', 'ruwet', 'gangguan', 'rusak', 'blank',
    # Masalah AI (Halusinasi)
    'salah', 'ngaco', 'bohong', 'sesat', 'tidak akurat', 'halu', 'halusinasi',
    'melenceng', 'tidak nyambung', 'gak nyambung', 'tipu', 'berbelit',
    # Masalah Akses (Penting untuk sentimen campuran)
    'bayar', 'berbayar', 'mahal', 'beli', 'langganan', 'premium', 'pro',
    'batas', 'terbatas', 'limit', 'dibatasi', 'kuota', 'habis', 'iklan',
    'terkunci', 'login susah'
]

# 2. Definisikan Fungsi Anotasi Hibrida
def anotasi_hibrida(row):
    # Handle error konversi jika masih ada data kotor
    try:
        score = int(row['score'])
    except:
        return 1 # Default ke Netral jika error

    ulasan = str(row['content']).lower()
    ada_positif = False
    ada_negatif = False

    # Pindai ulasan menggunakan Kamus
    for kata in KAMUS_POSITIF:
        if kata in ulasan:
            ada_positif = True
            break
    for kata in KAMUS_NEGATIF:
        if kata in ulasan:
            ada_negatif = True
            break

    # ATURAN HIBRIDA: Prioritas Utama
    # Jika mengandung unsur Positif DAN Negatif -> Paksa jadi Netral (1)
    if ada_positif and ada_negatif:
        return 1

    # ATURAN SKOR: Fallback
    if score <= 2:
        return 0 # Negatif
    elif score >= 4:
        return 2 # Positif
    else:
        return 1 # Netral (Biasa aja)

# 3. Terapkan Anotasi
print("--- Menerapkan Anotasi Hibrida pada Data Besar... ---")
df['label'] = df.apply(anotasi_hibrida, axis=1)

# 4. Finalisasi DataFrame
df_final = df[['content', 'label']].copy()
df_final.rename(columns={'content': 'text'}, inplace=True)
df_final.dropna(inplace=True)

# 5. Tampilkan Diagnosis Awal
print("\nDistribusi SEBELUM Balancing (Hasil Hibrida):")
# Menampilkan tabel diagnosis
label_counts_before = df_final['label'].value_counts().reset_index()
label_counts_before.columns = ['Label (Angka)', 'Jumlah Ulasan']
label_map = {0: 'Negatif', 1: 'Netral', 2: 'Positif'}
label_counts_before['Keterangan'] = label_counts_before['Label (Angka)'].map(label_map)
print(label_counts_before[['Keterangan', 'Label (Angka)', 'Jumlah Ulasan']].to_string(index=False))

# --- 6. Proses Balancing (Strategi Data Besar) ---
print("\n--- Memulai Balancing (Target: 3000 data per kelas)... ---")

df_pos = df_final[df_final['label'] == 2]
df_neg = df_final[df_final['label'] == 0]
df_neu = df_final[df_final['label'] == 1]

# TARGET DIUBAH: Kita naikkan ke 3000 karena data sumber kita 15.000
TARGET_COUNT = 3000

# A. Undersampling Positif (Mayoritas)
# replace=False -> Kita ambil data UNIK, tidak ada duplikasi (karena stok banyak)
df_pos_resampled = resample(df_pos,
                            replace=False,
                            n_samples=TARGET_COUNT,
                            random_state=42)

# B. Oversampling Negatif & Netral (Minoritas)
# replace=True -> Kita duplikasi data karena stok sedikit
df_neg_resampled = resample(df_neg,
                            replace=True,
                            n_samples=TARGET_COUNT,
                            random_state=42)

df_neu_resampled = resample(df_neu,
                            replace=True,
                            n_samples=TARGET_COUNT,
                            random_state=42)

# Gabungkan & Kocok
df_resampled = pd.concat([df_pos_resampled, df_neg_resampled, df_neu_resampled])
df_final = df_resampled.sample(frac=1, random_state=42).reset_index(drop=True)

print("--- Balancing Selesai. ---")

# 7. Tampilkan Hasil Akhir
print("\nDistribusi Label SETELAH Balancing (Data Seimbang & Kaya):")
label_counts_after = df_final['label'].value_counts().reset_index()
label_counts_after.columns = ['Label (Angka)', 'Jumlah Ulasan']
label_counts_after['Keterangan'] = label_counts_after['Label (Angka)'].map(label_map)
print(label_counts_after[['Keterangan', 'Label (Angka)', 'Jumlah Ulasan']].to_string(index=False))

--- Menerapkan Anotasi Hibrida pada Data Besar... ---

Distribusi SEBELUM Balancing (Hasil Hibrida):
Keterangan  Label (Angka)  Jumlah Ulasan
   Positif              2          12167
    Netral              1            597
   Negatif              0            430

--- Memulai Balancing (Target: 3000 data per kelas)... ---
--- Balancing Selesai. ---

Distribusi Label SETELAH Balancing (Data Seimbang & Kaya):
Keterangan  Label (Angka)  Jumlah Ulasan
    Netral              1           3000
   Positif              2           3000
   Negatif              0           3000


# **Tahap 3: Persiapan Modeling (Train-Test Split)**

In [ ]:
# --- Tahap 3: Persiapan Modeling (Split Data) ---

# 1. Bagi Data (80% Latih, 20% Validasi)
train_df, val_df = train_test_split(
    df_final,
    test_size=0.2,          # 20% dari data dialokasikan untuk validasi.
    random_state=42,        # Memastikan hasil pembagian data konsisten setiap kali dijalankan.
    stratify=df_final['label'] # Memastikan proporsi label (0,1,2) seimbang di kedua set.
)

print(f"Jumlah data latih: {len(train_df)}")
print(f"Jumlah data validasi: {len(val_df)}")

# 2. Konversi ke Format 'Dataset' Hugging Face
# Mengubah format Pandas DataFrame menjadi 'Dataset' agar kompatibel dengan Trainer.
train_dataset = Dataset.from_pandas(train_df)
val_dataset = Dataset.from_pandas(val_df)

print("\nKonversi ke format Hugging Face Dataset selesai.")
print(train_dataset)

Jumlah data latih: 7200
Jumlah data validasi: 1800

Konversi ke format Hugging Face Dataset selesai.
Dataset({
    features: ['text', 'label', '__index_level_0__'],
    num_rows: 7200
})


# **Tahap 4: Representasi Teks (Tokenisasi & Vectorization)**

In [ ]:
# --- Tahap 4: Representasi Teks (Tokenisasi & Vectorization) ---

# 1. Tentukan Nama Model
# Menggunakan 'indobenchmark/indobert-base-p1', model Transformer
# yang sudah dilatih secara ekstensif pada teks Bahasa Indonesia.
MODEL_NAME = "indobenchmark/indobert-base-p1"

# 2. Muat Tokenizer
# Tokenizer akan mengubah teks mentah (string) menjadi representasi angka (token ID)
# yang dipahami oleh model IndoBERT.
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

# 3. Definisikan Fungsi Tokenisasi
def tokenize_function(examples):
    # 'padding' dan 'truncation' memastikan semua input memiliki panjang yang seragam (128 token).
    return tokenizer(
        examples["text"],
        padding="max_length",
        truncation=True,
        max_length=128
    )

# 4. Terapkan Tokenisasi ke Seluruh Dataset
print("\n--- Memulai Tokenisasi Data Latih & Validasi ---")
# '.map()' menerapkan 'tokenize_function' ke semua baris data secara efisien.
tokenized_train_dataset = train_dataset.map(tokenize_function, batched=True)
tokenized_val_dataset = val_dataset.map(tokenize_function, batched=True)

# 5. Bersihkan Kolom yang Tidak Perlu
# Menghapus kolom 'text' (karena sudah diganti 'input_ids')
# dan metadata sisa dari pandas.
tokenized_train_dataset = tokenized_train_dataset.remove_columns(["text", "__index_level_0__"])
tokenized_val_dataset = tokenized_val_dataset.remove_columns(["text", "__index_level_0__"])

print("--- Tokenisasi Selesai ---")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]


--- Memulai Tokenisasi Data Latih & Validasi ---


Map:   0%|          | 0/7200 [00:00<?, ? examples/s]

Map:   0%|          | 0/1800 [00:00<?, ? examples/s]

--- Tokenisasi Selesai ---


# **Tahap 5: Pemuatan Model & Metrik (Modeling - Bagian 1)**

In [ ]:
# --- Tahap 5: Pemuatan Model & Metrik ---

# 1. Muat Model
# Memuat arsitektur IndoBERT dengan 'kepala' klasifikasi di atasnya.
# 'num_labels=3' menginformasikan model bahwa ada 3 kemungkinan output (0, 1, 2).
model = AutoModelForSequenceClassification.from_pretrained(
    MODEL_NAME,
    num_labels=3
)

# 2. Definisikan Pemetaan Label
# Ini membantu pipeline di akhir untuk memberi output "Positif" bukan "LABEL_2".
model.config.id2label = {0: "Negatif", 1: "Netral", 2: "Positif"}
model.config.label2id = {"Negatif": 0, "Netral": 1, "Positif": 2}

# 3. Definisikan Fungsi Metrik Evaluasi
# Fungsi ini akan dipanggil oleh Trainer untuk menghitung skor performa model.
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)

    # Menghitung Akurasi
    acc = accuracy_score(labels, preds)

    # Menghitung F1-Score dengan 'weighted' average,
    # yang ideal untuk dataset tidak seimbang (imbalanced dataset).
    f1 = f1_score(labels, preds, average='weighted')

    return {'accuracy': acc, 'f1': f1}

print("--- Tahap 5 Selesai: Model & Metrik siap. ---")

pytorch_model.bin:   0%|          | 0.00/498M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at indobenchmark/indobert-base-p1 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


--- Tahap 5 Selesai: Model & Metrik siap. ---


# **Tahap 6: Pelatihan Model (Modeling - Bagian 2)**

In [ ]:
# --- Tahap 6: Pelatihan Model (Fine-Tuning) dengan Label Smoothing ---

# 1. Definisikan Argumen Training
training_args = TrainingArguments(
    output_dir='./results',                # Direktori untuk menyimpan checkpoint model
    num_train_epochs=3,                    # Model akan "belajar" melewati data sebanyak 3 kali
    per_device_train_batch_size=16,        # Jumlah data per batch saat training
    per_device_eval_batch_size=16,         # Jumlah data per batch saat evaluasi
    warmup_steps=500,                      # Jumlah langkah pemanasan learning rate
    weight_decay=0.01,                     # Parameter regularisasi L2
    logging_dir='./logs',
    logging_steps=10,                      # Mencatat log training setiap 10 langkah

    # Argumen Kunci untuk Evaluasi:
    eval_strategy="epoch",                 # Melakukan evaluasi setiap akhir epoch
    save_strategy="epoch",                 # Menyimpan model setiap akhir epoch
    load_best_model_at_end=True,           # Otomatis memuat model terbaik di akhir training

    # LABEL SMOOTHING ---
    # Menerapkan label smoothing sebesar 0.1 untuk mengurangi overfitting
    # dan meningkatkan generalisasi pada kelas dengan data sedikit (Netral).
    label_smoothing_factor=0.1,

    report_to="none",                      # Menonaktifkan logging ke platform eksternal
)

# 2. Inisialisasi Trainer
# Trainer adalah objek yang mengelola proses training dan evaluasi.
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_dataset, # Data latih
    eval_dataset=tokenized_val_dataset,    # Data validasi
    compute_metrics=compute_metrics,       # Fungsi metrik
)

# 3. Mulai Training
print("\n--- MEMULAI TRAINING MODEL (SMOOTHING ENABLED) ---")
trainer.train()
print("--- TRAINING SELESAI ---")


--- MEMULAI TRAINING MODEL (SMOOTHING ENABLED) ---


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.612700,0.632537,0.818889,0.819888
2,0.539400,0.512571,0.888333,0.889596
3,0.495900,0.491206,0.898333,0.898992


--- TRAINING SELESAI ---


# **Tahap 7: Evaluasi Model (Modeling - Bagian 3)**

In [ ]:
# --- Tahap 7: Evaluasi Model ---

print("\n--- HASIL EVALUASI MODEL TERBAIK ---")
# Menjalankan evaluasi pada data validasi (data yang belum pernah dilihat)
eval_results = trainer.evaluate()

print(f"Akurasi: {eval_results['eval_accuracy']:.4f}")
print(f"F1-Score (Weighted): {eval_results['eval_f1']:.4f}")

# 2. Tampilkan Laporan Klasifikasi Detil
print("\n--- LAPORAN KLASIFIKASI DETIL ---")
predictions = trainer.predict(tokenized_val_dataset)
preds = np.argmax(predictions.predictions, axis=-1)
true_labels = tokenized_val_dataset["label"]

# 'classification_report' menampilkan Precision, Recall, F1-Score untuk setiap kelas.
print(classification_report(
    true_labels,
    preds,
    target_names=["Negatif (0)", "Netral (1)", "Positif (2)"]
))


--- HASIL EVALUASI MODEL TERBAIK ---


Akurasi: 0.8956
F1-Score (Weighted): 0.8963

--- LAPORAN KLASIFIKASI DETIL ---
              precision    recall  f1-score   support

 Negatif (0)       0.93      0.91      0.92       600
  Netral (1)       0.95      0.86      0.90       600
 Positif (2)       0.82      0.91      0.86       600

    accuracy                           0.90      1800
   macro avg       0.90      0.90      0.90      1800
weighted avg       0.90      0.90      0.90      1800



# **Tahap 8: Penyimpanan Model**

In [ ]:
# --- Tahap 8: Penyimpanan Model ---

# Definisikan nama folder untuk menyimpan model final
OUTPUT_MODEL_DIR = "./model_sentimen_chatgpt_final"

# Menyimpan model yang sudah dilatih dan tokenizer-nya
trainer.save_model(OUTPUT_MODEL_DIR)
tokenizer.save_pretrained(OUTPUT_MODEL_DIR)

print(f"\nModel dan tokenizer telah disimpan di folder '{OUTPUT_MODEL_DIR}'")


Model dan tokenizer telah disimpan di folder './model_sentimen_chatgpt_final'


# **Tahap 9: Aplikasi (Gradio UI)**

In [ ]:
# --- Tahap 9: Aplikasi (Gradio UI) ---

# 1. Muat Model yang Sudah Dilatih
print("--- Memuat model terlatih ke pipeline... ---")
# 'pipeline' adalah cara mudah dari Hugging Face untuk inferensi (prediksi)
sentiment_pipeline = pipeline(
    "sentiment-analysis",    # Jenis task
    model=OUTPUT_MODEL_DIR,  # Folder model kita
    tokenizer=OUTPUT_MODEL_DIR, # Folder tokenizer kita
    device=0 # Menggunakan GPU (ubah ke -1 jika ingin CPU)
)
print("--- Pipeline siap. ---")

# 2. Definisikan Fungsi Prediksi
# Fungsi ini akan dipanggil oleh Gradio setiap kali user menekan "Submit".
def predict_sentiment(text_input):
    # Membersihkan input
    if not text_input or not text_input.strip():
        return "Input kosong. Harap masukkan teks ulasan."

    # Melakukan prediksi menggunakan pipeline
    try:
        result = sentiment_pipeline(text_input)

        # Format output
        label = result[0]['label']
        score = result[0]['score']
        return f"Hasil Sentimen: {label} (Skor: {score:.4f})"
    except Exception as e:
        return f"Terjadi error: {str(e)}"

# 3. Rancang Antarmuka (Interface) Gradio
print("\n--- Meluncurkan Antarmuka Gradio... ---")
iface = gr.Interface(
    fn=predict_sentiment,                   # Fungsi yang akan dipanggil
    inputs=gr.Textbox(lines=3, placeholder="Tulis ulasan Anda di sini..."), # Komponen input
    outputs="text",                         # Komponen output
    title="Analisis Sentimen Ulasan ChatGPT (IndoBERT)",
    description="Demo UAS NLP. Masukkan ulasan tentang ChatGPT dalam Bahasa Indonesia untuk memprediksi sentimennya (Positif, Negatif, atau Netral)."
)

# 4. Luncurkan Aplikasi
# 'debug=True' untuk log error
# 'share=True' akan membuat link publik (berlaku 72 jam) untuk demo
iface.launch(debug=True, share=True)

Device set to use cuda:0


--- Memuat model terlatih ke pipeline... ---
--- Pipeline siap. ---

--- Meluncurkan Antarmuka Gradio... ---
Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://e3604a0b51f0c3634b.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://e3604a0b51f0c3634b.gradio.live


## **Code di bawah/ di dalam sel ini jangan di running atau di hapus ini backup code tahap 2**